# Binance

- Créér un clé API : https://www.binance.com/en/support/faq/360002502072
- **Quand elle est créée garder en mémoire l'api key et la secret KEY**

- créer un fichier config.cfg dans le même répertoire que le notebook et remplacer les valeurs par celles de l'API créée

```cfg
[BINANCE]
API_KEY = YOUR KEY                  # A MODIFIER
SECRET_KEY = YOUR SECRET KEY        # A MODIFIER
```

- installer le package python-binance 

```shell
pip install python-binance
ou
py -m pip install python-binance
ou autre
```
- Editer la list my_coins avec vos crypto

```python
#exemple
my_coins = ['BTC','USDT', 'ETH']     # A ADAPTER
```

- Editer le dict stack car l'API ne sait pas encore récupérer les crypto stackées 

```python
#exemple
stack = {'DOT':3, 'ADA':6.1938, 'CAKE':3.125, 'TRX': 67}      # A ADAPTER
```

- et lancer tout les cellules
! Tout est en dollars


In [ ]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
import requests, json, time, hashlib,hmac, configparser

# A ADAPTER
my_coins = ['BTC','USDT', 'ETH', 'ADA', 'DOT', 'CAKE', 'MATIC', 'VET', 'BNB', 'LTC', 'KSM', 'THETA', 'AVAX', 'DENT', 'EGLD', 'HOT', 'TRX', 'ONE', 'BAND', 'SXP', 'LINK']  
my_symbols = [coin + 'USDT' for coin in my_coins]

config = configparser.ConfigParser()
config.read_file(open('config.cfg'))

KEY = config.get('BINANCE', 'API_KEY')
SECRET_KEY = config.get('BINANCE', 'SECRET_KEY')
BASE_URL = 'https://api.binance.com/'

client = Client(KEY, SECRET_KEY)


In [ ]:
from urllib.parse import urlencode

# fonctions utilitaires

def get_timestamp():
    return int(time.time() * 1000)

def hashing(query_string):
    return hmac.new(SECRET_KEY.encode('utf-8'), query_string.encode('utf-8'), hashlib.sha256).hexdigest()

def dispatch_request(http_method):
    session = requests.Session()
    session.headers.update({
        'Content-Type': 'application/json;charset=utf-8',
        'X-MBX-APIKEY': KEY
    })
    return {
        'GET': session.get,
        'DELETE': session.delete,
        'PUT': session.put,
        'POST': session.post,
    }.get(http_method, 'GET')

def send_signed_request(http_method, url_path, payload={}):
    query_string = urlencode(payload, True)
    if query_string:
        query_string = "{}&timestamp={}".format(query_string, get_timestamp())
    else:
        query_string = 'timestamp={}'.format(get_timestamp())

    url = BASE_URL + url_path + '?' + query_string + '&signature=' + hashing(query_string)
    print("{} {}".format(http_method, url))
    params = {'url': url, 'params': {}}
    response = dispatch_request(http_method)(**params)
    return response.json()


In [ ]:
# Create wallet
def create_wallet():
    wallet = {}
    for asset in my_coins:
        wallet[asset] = {'Qty':0, 'Price':0, 'Saving_value':0, 'Spot_value':0, 'Stack_value':0, 'Balance':0}
    return wallet

In [ ]:
# Get order history
def update_order_history():
    for symbol in my_symbols:
        if symbol != 'USDTUSDT':
            orders_history = client.get_all_orders(symbol=symbol)
            sum_coin = 0.0
            sum_cost = 0.0
            for order in orders_history:
                sum_cost += float(order['executedQty']) * float(order['price'])
                sum_coin += float(order['executedQty']) 
            wallet[symbol.replace('USDT', '')]['Qty'] = sum_coin
            wallet[symbol.replace('USDT', '')]['Price'] =sum_cost

In [ ]:
# Get current Savings
def update_saving():
    saving = send_signed_request('GET', 'sapi/v1/lending/union/account')
    savings = saving['positionAmountVos']
    for asset in savings:
        wallet[asset['asset']]["Saving_value"] = float(asset['amountInUSDT'])

In [ ]:
# stop value
def update_spot():
    all_coins = client.get_all_tickers()
    spot = send_signed_request('GET', 'api/v3/account')

    for asset in my_coins:
        for asset_spot in spot["balances"]:
            if asset_spot['asset'] == asset:
                for symbol in all_coins:
                    if symbol['symbol'] == asset+'USDT':
                        wallet[asset]['Spot_value'] = float(symbol['price']) * float(asset_spot['free'])

In [ ]:
# get stacking value
def update_stacking():    
    all_coins = client.get_all_tickers()
    # A ADAPTER
    stack = {'DOT':2.50085407+1.12174, 'ADA':6.1938, 'CAKE':3.125, 'TRX': 67}
    for asset in my_coins:
        try:
            coin = stack[asset]
            for symbol in all_coins:
                if symbol['symbol'] == asset+'USDT':
                    wallet[asset]['Stack_value'] = float(symbol['price']) * coin
        except:
            pass

In [ ]:
# balance calcul
def update_balance():
    for asset in my_coins:
        wallet[asset]["Balance"] =  wallet[asset]["Stack_value"] + wallet[asset]["Saving_value"] + wallet[asset]["Spot_value"] -  wallet[asset]["Price"]

In [ ]:
wallet = create_wallet()
update_order_history()
update_saving()
update_spot()
update_stacking()
update_balance()

wallet

In [ ]:
## Bilan

# global balance
balance = 0
total_usdt_invest = 0
total_spot = 0
total_saving = 0
total_stacking = 0
current_value = 0

for asset in wallet:
    if asset != "USDT":
        balance += wallet[asset]['Balance']
        total_usdt_invest +=  wallet[asset]['Price']
        total_spot += wallet[asset]['Spot_value']
        total_saving += wallet[asset]['Saving_value']
        total_stacking += wallet[asset]['Stack_value']
    else:
        total_usdt_invest +=wallet[asset]['Balance']
        total_spot += wallet[asset]['Spot_value']
        total_saving += wallet[asset]['Saving_value']
        total_stacking += wallet[asset]['Stack_value']

current_value = total_spot + total_saving + total_stacking

print('total_usdt_invest',total_usdt_invest,'$')
print('current_value',current_value,'$')
print('balance',balance,'$')
print('total_spot',total_spot,'$')
print('total_saving',total_saving,'$')
print('total_stacking',total_stacking,'$')